In [1]:
import pandas as pd

### Finding the coffee production schedule

In [2]:
unformatted_coffee_production_seasons = pd.read_csv("../datasets/capstone/downloaded data/Coffee Producing Countries and Their Harvest Seasons.csv")

coffee_production_seasons = pd.DataFrame()

for i, row in enumerate(unformatted_coffee_production_seasons.index):
    if unformatted_coffee_production_seasons["Producing Country"].isnull().loc[row]:
        unformatted_coffee_production_seasons.loc[row, "Producing Country"] = unformatted_coffee_production_seasons.ix[i - 1, "Producing Country"]

coffee_production_seasons = pd.concat(objs = [unformatted_coffee_production_seasons, 
                  unformatted_coffee_production_seasons["Harvest Season"].str.split("-", expand = True)
                 ], axis = 1)

coffee_production_seasons.columns = ["Producing Country", 
                                     "Coffee Type", 
                                     "Harvest Season", 
                                     "Harvest Begins", 
                                     "Harvest Ends", 
                                     "Year-round Harvest"]

# Correct the misspelling of Colombia
coffee_production_seasons.ix[14, "Producing Country"] = "Colombia"

for row in coffee_production_seasons[~ coffee_production_seasons["Year-round Harvest"].isnull()].index:
    coffee_production_seasons.iloc[row, 3] = coffee_production_seasons.iloc[row, 4].replace("Peak ", "")
    coffee_production_seasons.iloc[row, 4] = coffee_production_seasons.iloc[row, 5]
    coffee_production_seasons.iloc[row, 5] = True
    
coffee_production_seasons.loc[:, "Year-round Harvest"] = coffee_production_seasons["Year-round Harvest"].map({None: False, True: True})

coffee_production_seasons.drop("Harvest Season", axis = 1, inplace = True)

coffee_production_seasons["Harvest Begins"] = coffee_production_seasons["Harvest Begins"].str.strip()
coffee_production_seasons["Harvest Ends"] = coffee_production_seasons["Harvest Ends"].str.strip()

# Impute ending months for Sri Lanka harvest based on harvest duration of similar countries (Ethiopia and Vietnam)
coffee_production_seasons.ix[54:55, "Harvest Ends"] = "October"

# Robusta harvest months for Panama, Papua New Guinea, Kenya, Colombia, Peru, Costa Rica, and Burundi, 
# but robusta production in these countries is negligible

import calendar
month_to_integer_dictionary = {name: num for num, name in enumerate(calendar.month_name) if num}
coffee_production_seasons["Harvest Begins"] = coffee_production_seasons["Harvest Begins"].map(month_to_integer_dictionary)
coffee_production_seasons["Harvest Ends"] = coffee_production_seasons["Harvest Ends"].map(month_to_integer_dictionary)

coffee_production_seasons = pd.concat([coffee_production_seasons, pd.get_dummies(coffee_production_seasons["Coffee Type"])], axis = 1)
coffee_production_seasons["Arabica"] = coffee_production_seasons["Arabica"].map({1: True, 0: False})
coffee_production_seasons["Robusta"] = coffee_production_seasons["Robusta"].map({1: True, 0: False})
coffee_production_seasons.drop("Coffee Type", axis = 1, inplace = True)


In [3]:
coffee_production_seasons.to_csv("../datasets/capstone/coffee harvest schedule.csv")